In [1918]:
# class Word defines a word in dictionary; 
class Word(object):
    def __init__(self, WordName=None, WordIndex=None, WordFrequency=None):
        self.WordName = WordName
        self.WordIndex = WordIndex
        self.WordFrequency = WordFrequency


# class Element defines a word in a bug corpus or in a code corpus; 
class Element(object):
    def __init__(self, ElementName=None, ElementIndex=None, ElementFrequency=None):
        self.ElementName = ElementName
        self.ElementIndex = ElementIndex
        self.ElementFrequency = ElementFrequency
        self.ElementWeight = 0


# class Corpus defines a bug corpus or a code corpus; 
class BugCorpus(object):
    def __init__(self, FileName=None):
        self.FileName = FileName
        self.Norm = 0
        self.Similarity = 0
        self.CorpusMembers = []
        self.RelevantCodes = []
        
# class Corpus defines a bug corpus or a code corpus; 
class CodeCorpus(object):
    def __init__(self, FileName=None):
        self.FileName = FileName
        self.Nx = 0
        self.Norm = 0
        self.rVSMScore = 0
        self.CorpusMembers = []
        self.RelevantBugs = []
        self.SimiScore = 0
        self.FinalScore = 0
        self.Rank = 0

In [1919]:
def generate_OneBugCorpus(FilePath, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    with open(FilePath) as TextFile:
        ThisBugFile = TextFile.read().split(' ')    


    ThisBugCorpus = BugCorpus(FilePath) # this file delete word that is not in the word list; 

    for i in range(len(ThisBugFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisBugFile[i] == WordList[j].WordName:
                flag = True
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break        
               

    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisBugCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency))            
        
    return ThisBugCorpus

In [1920]:
def generate_OneCodeCorpus(ThisCodeFile, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    ThisCodeCorpus = CodeCorpus(ThisCodeFile[0])
    
    for i in range(len(ThisCodeFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisCodeFile[i] == WordList[j].WordName:
                flag = True                    
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break
    
    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisCodeCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency))     
    
    return ThisCodeCorpus

In [1921]:
def compute_BugCorpus_weight(CorpusNumber, MemberNumber):
    
    ThisWord = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementName
    # count ftd, the frequency of term t in document d;
    ftd = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementFrequency
#    print("ftd:", ftd)

    # #docs, total number of documents; 
    NumDocs = len(BugFiles) + len(CodeCorpuses);
#    print("#docs:", NumDocs)

    # count nt, the number of docuement that contains t;
    # count ntBug, the number of document that contains t in bug files; 
    ntBug = 0 
    for i in range(len(BugCorpuses)):    
        flag = False        
        for j in range(len(BugCorpuses[i].CorpusMembers)):
            if ThisWord == BugCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
            ntBug = ntBug + 1

#    print("ntBug:", ntBug)

    # count ntCode, the number of document that contains t in code files; 
    ntCode = 0
    for i in range(len(CodeCorpuses)):
        flag = False
        for j in range(len(CodeCorpuses[i].CorpusMembers)):
            if ThisWord == CodeCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
                ntCode = ntCode + 1

#    print("ntCode:", ntCode)

    nt = ntBug + ntCode
#    print("nt:", nt)

    weight = (math.log(ftd)+1) * math.log(NumDocs/nt)
#    print(weight)
    
    return weight

In [1922]:
def compute_CodeCorpus_weight(CorpusNumber, MemberNumber):
    
    ThisWord = CodeCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementName
    # count ftd, the frequency of term t in document d;
    ftd = CodeCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementFrequency
#    print("ftd:", ftd)

    # #docs, total number of documents; 
    NumDocs = len(BugFiles) + len(CodeCorpuses);
#    print("#docs:", NumDocs)

    # count nt, the number of docuement that contains t;
    # count ntBug, the number of document that contains t in bug files; 
    ntBug = 0 
    for i in range(len(BugCorpuses)):    
        flag = False        
        for j in range(len(BugCorpuses[i].CorpusMembers)):
            if ThisWord == BugCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
            ntBug = ntBug + 1

#    print("ntBug:", ntBug)

    # count ntCode, the number of document that contains t in code files; 
    ntCode = 0
    for i in range(len(CodeCorpuses)):
        flag = False
        for j in range(len(CodeCorpuses[i].CorpusMembers)):
            if ThisWord == CodeCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
                ntCode = ntCode + 1

#    print("ntCode:", ntCode)

    nt = ntBug + ntCode
#    print("nt:", nt)

    weight = (math.log(ftd)+1) * math.log(NumDocs/nt)
#    print(weight)
    
    return weight

In [1923]:
def compute_BugCorpus_norm(CorpusNumber):
    norm = 0
    for i in range(len(BugCorpuses[CorpusNumber].CorpusMembers)):
        norm = norm + BugCorpuses[CorpusNumber].CorpusMembers[i].ElementWeight**2
        
    norm = norm**0.5
    
    return norm

In [1924]:
def compute_CodeCorpus_norm(CorpusNumber):
    norm = 0
    for i in range(len(CodeCorpuses[CorpusNumber].CorpusMembers)):
        norm = norm + CodeCorpuses[CorpusNumber].CorpusMembers[i].ElementWeight**2
        
    norm = norm**0.5
    
    return norm

In [1925]:
def compute_BugCode_intersection(BugCorpusNumber, CodeCorpusNumber):
    intersection = 0
    for i in range(len(BugCorpuses[BugCorpusNumber].CorpusMembers)):
        for j in range(len(CodeCorpuses[CodeCorpusNumber].CorpusMembers)):
            if BugCorpuses[BugCorpusNumber].CorpusMembers[i].ElementName == CodeCorpuses[CodeCorpusNumber].CorpusMembers[j].ElementName:
                intersection += BugCorpuses[BugCorpusNumber].CorpusMembers[i].ElementWeight * CodeCorpuses[CodeCorpusNumber].CorpusMembers[j].ElementWeight
                break
    
    return intersection    

In [1926]:
def compute_BugBug_intersection(TargetBugNumber, OtherBugNumber):
    intersection = 0
    for i in range(len(BugCorpuses[TargetBugNumber].CorpusMembers)):
        for j in range(len(BugCorpuses[OtherBugNumber].CorpusMembers)):
            if BugCorpuses[TargetBugNumber].CorpusMembers[i].ElementName == BugCorpuses[OtherBugNumber].CorpusMembers[j].ElementName:
                intersection += BugCorpuses[TargetBugNumber].CorpusMembers[i].ElementWeight * BugCorpuses[OtherBugNumber].CorpusMembers[j].ElementWeight
                break
    
    return intersection    

In [1927]:
def compute_rVSMScore(BugCorpusNumber, CodeCorpusNumber):
    intersection = compute_BugCode_intersection(BugCorpusNumber, CodeCorpusNumber)
    g = 1 / (1 + math.exp(-CodeCorpuses[CodeCorpusNumber].Nx))
    rVSM_score = g * intersection / (BugCorpuses[BugCorpusNumber].Norm * CodeCorpuses[CodeCorpusNumber].Norm)
    
    
    return rVSM_score

In [1928]:
def compute_Similarity(TargetBugNumber, OtherBugNumber):
    intersection = compute_BugBug_intersection(TargetBugNumber, OtherBugNumber)
    Similarity = intersection / (BugCorpuses[TargetBugNumber].Norm * BugCorpuses[OtherBugNumber].Norm)
    
    return Similarity

In [1929]:
def compute_SimiScore(CodeNumber, BugFileNames):
    SimiScore = 0
    counter = 0
    for i in range(len(CodeCorpuses[CodeNumber].RelevantBugs)):
        ThisBugName = CodeCorpuses[CodeNumber].RelevantBugs[i]
        ThisBugNumber = BugFileNames.index(ThisBugName)
        
        if BugCorpuses[ThisBugNumber].Similarity > 0:
            SimiScore += BugCorpuses[ThisBugNumber].Similarity
            counter += 1
    
    if counter != 0:
        SimiScore = SimiScore / counter
    
    return SimiScore

In [1930]:
# this process intends to save all file names in an array; 

import os

FolderPath = 'C:\\Users\\take_\\OneDrive\\liming\\where\\BugLocator\\tmp\\'
path = FolderPath + 'BugCorpus\\'

BugFiles = []
# r=root, d=directories, f = bug files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            BugFiles.append(os.path.join(r, file))

print(len(BugFiles))

98


In [1931]:

## ----------------------------------- read word list from Wordlist.txt --------------------------------------------
with open(FolderPath + "WordList.txt") as TextFile:
    WordlistContent = [line.split() for line in TextFile]

WordList = []
    
for i in range(len(WordlistContent)):    
    WordList.append(Word(WordlistContent[i][0], WordlistContent[i][1], 0))
    
# test whether WordList is correct or not;     
#print(WordList[0].WordName)
#print(WordList[1].WordName)
    
    
## ----------------------------------- read all bug files ------------------------------------------------    
#ThisBugCorpus = generate_OneBugCorpus(BugFiles[0], WordList)
#print(ThisBugCorpus.FileName)
#print(ThisBugCorpus.CorpusMembers)

BugCorpuses = []
for i in range(len(BugFiles)):
    ThisBugCorpus = generate_OneBugCorpus(BugFiles[i], WordList)
    BugCorpuses.append(ThisBugCorpus)

# test result of all bug corpuses;     
# print(BugCorpuses[0])
for i in range(1):
    print(BugCorpuses[i].FileName)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
    print()

## ----------------------------------- read all code files ------------------------------------------------
with open(FolderPath + "CodeCorpus.txt") as TextFile:
    CodeList = TextFile.read().split()   

#print(len(CodeList))
CodeFiles = []
ThisCodeFile = []

for i in range(len(CodeList)):
    if CodeList[i][-5:] == ".java":        
        CodeFiles.append(ThisCodeFile)
        ThisCodeFile = []
        ThisCodeFile.append(CodeList[i])
    else:
        ThisCodeFile.append(CodeList[i])

CodeFiles.append(ThisCodeFile)
CodeFiles.pop(0) # delete the first element;         

#print(len(CodeFiles))
#print(CodeFiles[0])

#ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[0], WordList)
#print(ThisCodeCorpus)

# generate code corpuses from code files and find maximum and minimum length of code document;
CodeCorpuses = []
MinDocumentLength = len(CodeFiles[0])
MaxDocumentLength = len(CodeFiles[0])
for i in range(len(CodeFiles)):
    ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[i], WordList)
    CodeCorpuses.append(ThisCodeCorpus)
    
    if len(CodeFiles[i]) < MinDocumentLength:
        MinDocumentLength = len(CodeFiles[i])
    if len(CodeFiles[i]) > MaxDocumentLength:
        MaxDocumentLength = len(CodeFiles[i])
    
#print(MinDocumentLength)
#print(MaxDocumentLength)    
    
# compute Nx for each code file;
for i in range(len(CodeCorpuses)):
    CodeCorpuses[i].Nx = (len(CodeFiles[i]) - MinDocumentLength) / (MaxDocumentLength - MinDocumentLength)
#    print(CodeCorpuses[i].Nx)
    
# test all code corpuses generated; 
print(CodeCorpuses[0].FileName)
#print(CodeCorpuses[0].CorpusMembers)
for i in range(len(CodeCorpuses[0].CorpusMembers)):
    print(CodeCorpuses[0].CorpusMembers[i].ElementName, end = ',')
    print(CodeCorpuses[0].CorpusMembers[i].ElementFrequency, end = ',')
print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100040.txt
averag,2,time,1,mirror,1,icon,2,includ,1,ilc,1,menu,1,perspect,3,test,1,numer,1,swt,2,ui,3,os,1,ms,6,number,1,window,1,increas,1,show,1,toolbar,1,run,1,perform,1,eclips,2,imagelist,2,make,1,elaps,6,process,6,singl,1,due,1,debug,4,build,1,
org.eclipse.swt.accessibility.ACC.java
client,1,access,1,scrollbar,1,radiobutton,1,listitem,1,acc,3,delet,1,deprec,1,multipl,1,tablecel,1,state,17,press,1,tabitem,1,insert,1,tablerowhead,1,focus,2,contain,1,combobox,1,dialog,1,tooltip,1,pushbutton,1,select,2,label,1,expand,1,separ,1,sizeabl,1,constant,1,hottrack,1,offscreen,1,treeitem,1,childid,3,multiselect,1,progressbar,1,menu,1,text,3,slider,1,readonli,1,tablecolumnhead,3,tree,1,menubar,1,checkbutton,1,check,1,defin,1,list,1,role,32,area,1,link,2,tablecolumn,1,sinc,1,xd,1,xc,1,xb,1,xa,1,us,2,invis,1,window,1,toolbar,1,object,1,tabfold,1,tabl,1,menuitem,1,busi,1,collaps,1,


In [1932]:
import math

#compute weight for each element in bug corpuses
for i in range(len(BugCorpuses)):
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        BugCorpuses[i].CorpusMembers[j].ElementWeight= compute_BugCorpus_weight(i, j)
    
# compute norm for each document in bug corpuses;
for i in range(len(BugCorpuses)):
    BugCorpuses[i].Norm = compute_BugCorpus_norm(i)

# test result of element weight;
for i in range(2):
    print(BugCorpuses[i].FileName)
    print(BugCorpuses[i].Norm)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print("[", end = '')
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementWeight, end = '')
        print("]", end = ',')
    print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100040.txt
33.12253336674247
[averag,2,8.054356224970617],[time,1,2.3774864011671637],[mirror,1,3.3219480100080148],[icon,2,5.078044048041838],[includ,1,2.9324832432462915],[ilc,1,4.757032535297338],[menu,1,2.454447442303292],[perspect,3,9.983166936169008],[test,1,3.034265937556234],[numer,1,3.8815637979434374],[swt,2,4.372181479321822],[ui,3,6.6055807552864545],[os,1,1.546188882126401],[ms,6,14.706592898115316],[number,1,1.9844438130575563],[window,1,1.722079548590065],[increas,1,3.6584202466292277],[show,1,1.972021293058999],[toolbar,1,3.968575174933067],[run,1,2.3961785341793163],[perform,1,3.1083739097099556],[eclips,2,3.96385862276427],[imagelist,2,8.054356224970617],[make,1,2.495269436823547],[elaps,6,13.903453748220905],[process,6,6.852226888772795],[singl,1,2.6052703320378754],[due,1,4.169245870395218],[debug,4,11.351679914643896],[build,1,3.4220314685649975],
C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCo

In [1933]:
#compute weight for each element in the code corpuses; 
for i in range(len(CodeCorpuses)):
    for j in range(len(CodeCorpuses[i].CorpusMembers)):
        CodeCorpuses[i].CorpusMembers[j].ElementWeight= compute_CodeCorpus_weight(i, j)

# compute norm for each document in code corpuses;
for i in range(len(CodeCorpuses)):
    CodeCorpuses[i].Norm = compute_CodeCorpus_norm(i)        
        
# test result of element weight;
for i in range(2):
    print(CodeCorpuses[i].FileName)
    print(CodeCorpuses[i].Norm)
    for j in range(len(CodeCorpuses[i].CorpusMembers)):
        print("[", end = '')
        print(CodeCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(CodeCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
        print(CodeCorpuses[i].CorpusMembers[j].ElementWeight, end = '')
        print("]", end = ',')
    print()

org.eclipse.swt.accessibility.ACC.java
38.42882752968844
[client,1,1.812593556130897],[access,1,1.6569402464191036],[scrollbar,1,3.968575174933067],[radiobutton,1,4.757032535297338],[listitem,1,5.267858159063328],[acc,3,7.9779188756714055],[delet,1,2.5163228460213793],[deprec,1,3.6584202466292277],[multipl,1,2.9652730660692823],[tablecel,1,5.673323267171493],[state,17,6.351405462877503],[press,1,2.434644815007112],[tabitem,1,4.757032535297338],[insert,1,2.65289838102713],[tablerowhead,1,5.673323267171493],[focus,2,6.88075603049247],[contain,1,1.2975662455112063],[combobox,1,4.757032535297338],[dialog,1,2.999174617744964],[tooltip,1,3.7274131181161794],[pushbutton,1,4.757032535297338],[select,2,2.6734433424492567],[label,1,3.370738174177447],[expand,1,2.9652730660692823],[separ,1,2.8111223862420243],[sizeabl,1,5.267858159063328],[constant,1,1.9356536488881242],[hottrack,1,5.267858159063328],[offscreen,1,4.574710978503383],[treeitem,1,4.757032535297338],[childid,3,7.677605686688315],[mul

In [1992]:
LocateBugNumber = 0

for LocateBugNumber in range(len(BugCorpuses)):
    ## This cell compute rVSMScore of all code corpuses with target bug and normalize them; 
    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].rVSMScore = compute_rVSMScore(LocateBugNumber, i)

    # find maximum rVSMScore and minimum rVSMScore; 
    MinrVSMScore = CodeCorpuses[0].rVSMScore
    MaxrVSMScore = CodeCorpuses[0].rVSMScore
    for i in range(len(CodeCorpuses)):
        if CodeCorpuses[i].rVSMScore < MinrVSMScore:
            MinrVSMScore = CodeCorpuses[i].rVSMScore
        if CodeCorpuses[i].rVSMScore > MaxrVSMScore:
            MaxrVSMScore = CodeCorpuses[i].rVSMScore

    # normalize rVSMScore for all code files; 
    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].rVSMScore = (CodeCorpuses[i].rVSMScore - MinrVSMScore) / (MaxrVSMScore - MinrVSMScore)
    # compute target bug similarity with all bug corpuses
    for i in range(len(BugCorpuses)):
        BugCorpuses[i].Similarity = compute_Similarity(LocateBugNumber, i)
    BugCorpuses[LocateBugNumber].Similarity = -1

    #  read bug-code relation from FixLink.txt 
    with open(FolderPath + "FixLink.txt") as TextFile:
        BugCode = [line.split() for line in TextFile]

    # create bug file name list so as to find index;
    BugFileNames = []
    for i in range(len(BugCorpuses)):
        BugFileNames.append(BugCorpuses[i].FileName)
        BugCorpuses[i].RelevantCodes = [] # clear the RelevantCodes; 

    # create code file name list so as to find index;
    CodeFileNames = []
    for i in range(len(CodeCorpuses)):
        CodeFileNames.append(CodeCorpuses[i].FileName)
        CodeCorpuses[i].RelevantBugs = [] # clear the RelevantBugs;     

    for i in range(len(BugCode)):
        ThisBugName = FolderPath + 'BugCorpus\\' + BugCode[i][0] + ".txt"
        ThisCodeName = BugCode[i][1]

        BugNumber = BugFileNames.index(ThisBugName)
        CodeNumber = CodeFileNames.index(ThisCodeName)

        BugCorpuses[BugNumber].RelevantCodes.append(ThisCodeName)
        CodeCorpuses[CodeNumber].RelevantBugs.append(ThisBugName)

    #print(CodeCorpuses[3].RelevantBugs)
    #print(BugCorpuses[0].RelevantCodes)

    #compute_SimiScore(3, BugFileNames)

    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].SimiScore = compute_SimiScore(i, BugFileNames)

    # find maximum SimiScore and minimum SimiScore; 
    MinSimiScore = CodeCorpuses[0].SimiScore
    MaxSimiScore = CodeCorpuses[0].SimiScore
    for i in range(len(CodeCorpuses)):
        if CodeCorpuses[i].SimiScore < MinSimiScore:
            MinSimiScore = CodeCorpuses[i].SimiScore
        if CodeCorpuses[i].SimiScore > MaxSimiScore:
            MaxSimiScore = CodeCorpuses[i].SimiScore

    # normalize rVSMScore for all code files; 
    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].SimiScore = (CodeCorpuses[i].SimiScore - MinSimiScore) / (MaxSimiScore - MinSimiScore)

    # compute final score for all code files;
    alpha = 0.2
    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].FinalScore = CodeCorpuses[i].rVSMScore * (1-alpha) + CodeCorpuses[i].SimiScore * alpha

    CodeCorpuses.sort(key=lambda x:x.FinalScore, reverse=True)

    for i in range(len(CodeCorpuses)):
        CodeCorpuses[i].Rank = i

    CodeFileNames = []
    for i in range(len(CodeCorpuses)):
        CodeFileNames.append(CodeCorpuses[i].FileName)

    for i in range(len(BugCorpuses[LocateBugNumber].RelevantCodes)):
        ThisCodeName = BugCorpuses[LocateBugNumber].RelevantCodes[i]
        ThisCodeNumber = CodeFileNames.index(ThisCodeName)
        print(BugCorpuses[LocateBugNumber].FileName.replace(FolderPath+'BugCorpus\\', ''), end = ' ')
        print(CodeCorpuses[ThisCodeNumber].Rank, end = ' ')
        print(CodeCorpuses[ThisCodeNumber].FileName, end = ' ')
        print(CodeCorpuses[ThisCodeNumber].FinalScore)

100040.txt 11 org.eclipse.swt.widgets.Button.java 0.3236725273833566
100040.txt 1 org.eclipse.swt.widgets.Display.java 0.5134293714587015
100040.txt 2 org.eclipse.swt.widgets.ImageList.java 0.4716699809350664
100040.txt 84 org.eclipse.swt.widgets.Menu.java 0.13737924321267178
100040.txt 92 org.eclipse.swt.widgets.TabFolder.java 0.12838317702852198
100040.txt 13 org.eclipse.swt.widgets.Table.java 0.30551431584979727
100040.txt 143 org.eclipse.swt.widgets.ToolItem.java 0.08349955785626616
100040.txt 19 org.eclipse.swt.widgets.Tree.java 0.2871473713163523
100095.txt 0 org.eclipse.swt.dnd.URLTransfer.java 0.8
100387.txt 0 org.eclipse.swt.custom.CTabFolder.java 1.0
101354.txt 2 org.eclipse.swt.graphics.ImageDataLoader.java 0.6668480082818078
102756.txt 6 org.eclipse.swt.widgets.Combo.java 0.7098661395125669
102794.txt 2 org.eclipse.swt.layout.GridLayout.java 0.6037894755296613
103863.txt 22 org.eclipse.swt.SWT.java 0.4313408200784332
103863.txt 3 org.eclipse.swt.widgets.Composite.java 0.620